<img src="img/mmselfsup_logo.png">

# 模型自监督预训练 之 MAE

<a href="https://colab.research.google.com/github/wangruohui/OpenMMLabCourse/blob/main/codes/MMSelfSup_tutorials/【7】模型自监督预训练%20之%20MAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMSelfSup Repo**：[https://github.com/open-mmlab/mmselfsup](https://github.com/open-mmlab/mmselfsup)

**MMSelfSup 官方文档链接**：[https://mmselfsup.readthedocs.io/en/latest](https://mmselfsup.readthedocs.io/en/latest)

**作者**：OpenMMLab

## 0. 自监督预训练方法介绍：Masked Autoencoders (MAE)

**论文地址**：https://arxiv.org/pdf/2111.06377.pdf

**MAE 基本思想**：将输入图像分块 `patch`，随机遮挡住一部分图像块。将未被遮挡住的图像块和对应的位置信息一同输入到模型中，让模型恢复被遮挡的图像块内容。

<img src="img/MAE.png">

## 1. 环境配置

### 1.1 查看 Python、PyTorch 和 Torchvision 的版本

In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!gcc --version

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### 1.2 安装 MMSelfSup 的依赖库：MMCV

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv

### 1.3  安装 MMSelfSup

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd /content/mmselfsup

In [ ]:
!git checkout dev

In [ ]:
# Install MMSelfSup from source
!pip install -e . 

### 1.4 检查安装是否正确

In [ ]:
import mmselfsup
print(mmselfsup.__version__)

## 2. 准备数据集

### 2.0 数据集介绍

本教程将在 `Tiny ImageNet` 数据集上训练 Masked Autoencoders (MAE) 模型。

Tiny ImageNet 数据集是 ImageNet 的一个子集。

该数据集包含 200 个类别，每个类别有 500 张训练图片、50 张验证图片和 50 张测试图片，共 120,000 张图像。每张图片均为 64×64 彩色图片。

数据集官方下载地址：http://cs231n.stanford.edu/tiny-imagenet-200.zip

### 2.1 下载数据集

使用 GNU [Wget](https://www.gnu.org/software/wget/) 工具从斯坦福官方网站下载：http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [ ]:
%cd /content/mmselfsup

In [ ]:
!mkdir data
%cd data
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

### 2.2 解压数据集

In [ ]:
!unzip -q tiny-imagenet-200.zip

In [ ]:
!rm -rf tiny-imagenet-200.zip

### 2.3 查看数据集目录

In [ ]:
# Check data directory
!apt-get install tree
!tree -d /content/mmselfsup/data

### 2.4 准备标注文件

为了减少大家重写 `加载数据集` 代码的负担，我们整理好了标注文件，复制到数据集根目录 `mmselfsup/data/tiny-imagenet-200` 下即可。

In [ ]:
%cd /content/mmselfsup/data

In [ ]:
!wget https://raw.githubusercontent.com/wangruohui/OpenMMLabCourse/main/codes/MMSelfSup_tutorials/anno_files/train.txt -P tiny-imagenet-200
!wget https://raw.githubusercontent.com/wangruohui/OpenMMLabCourse/main/codes/MMSelfSup_tutorials/anno_files/val.txt -P tiny-imagenet-200

## 3. 写模型自监督预训练的配置文件

1. 新建一个名为 `mae_vit-base-p16_1xb32-coslr-1e_tinyin200.py` 的配置文件。（配置文件命名要求 & 含义可参考[这里](https://mmsegmentation.readthedocs.io/zh_CN/latest/tutorials/config.html#id3))



2. 写训练配置文件的内容：
    1. 继承 [mae_vit-base-p16_8xb512-coslr-400e_in1k.py](https://github.com/open-mmlab/mmselfsup/blob/master/configs/selfsup/mae/mae_vit-base-p16_8xb512-coslr-400e_in1k.py) 配置文件
    2. 根据需求修改参数 samples_per_gpu（单个 GPU 的 Batch size）和 workers_per_gpu （单个 GPU 分配的数据加载线程数）
    3. 修改数据集路径和数据标注文件路径
    4. 根据 batch size 调整学习率（调整原则请参考：[这里](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#id2)）
    5. 修改训练的总轮数 epoch

In [ ]:
%cd /content/mmselfsup

In [ ]:
%%writefile /content/mmselfsup/configs/selfsup/mae/mae_vit-base-p16_1xb32-coslr-1e_tinyin200.py

_base_ = 'mae_vit-base-p16_8xb512-coslr-400e_in1k.py'

# dataset
data = dict(
    samples_per_gpu=32, 
    workers_per_gpu=2,
    train=dict(
        data_source=dict(
            data_prefix='data/tiny-imagenet-200/train',
            ann_file='data/tiny-imagenet-200/train.txt',
        )
    )
)

# optimizer
optimizer = dict(
    lr=1.5e-4 * 4096 / 256 * (32 / 512 * 8),
)

runner = dict(max_epochs=1)

## 4. 模型自监督预训练

我们推荐使用分布式训练工具 [tools/dist_train.sh](https://github.com/open-mmlab/mmselfsup/blob/master/tools/dist_train.sh) 来启动训练任务（即使您只用一张 GPU 进行训练）。
因为一些自监督预训练算法需要用多张 GPU 进行训练，为此 MMSelfSup 支持了多卡训练可能会用到的模块，如 `SyncBN` 等。如果算法在训练的过程中使用到了这些模块，但不使用分布式训练，就会报错。

```shell
bash tools/dist_train.sh ${CONFIG_FILE} ${GPUS} --work-dir ${YOUR_WORK_DIR} [optional arguments]
```

参数:
+ CONFIG_FILE：自监督训练的配置文件所在路径

+ GPUS：进行训练时所使用的 GPU 数量

+ work-dir：训练过程中产生模型和日志等文件的保存路径

其他可选参数 `optional arguments` 可参考[这里](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#id3)。

In [ ]:
%cd /content/mmselfsup

In [ ]:
!bash tools/dist_train.sh \
configs/selfsup/mae/mae_vit-base-p16_1xb32-coslr-1e_tinyin200.py \
1 \
--work_dir work_dirs/selfsup/mae/mae_vit-base-p16_1xb32-coslr-1e_tinyin200/ 

## 5. MAE 模型恢复被遮挡图片效果的可视化

### 5.1 准备自监督预训练模型文件

In [ ]:
%cd /content/mmselfsup
!mkdir checkpoints
!wget -nc https://download.openmmlab.com/mmselfsup/mae/mae_visualize_vit_large.pth -P checkpoints

### 5.2 准备自监督预训练模型的配置文件

In [ ]:
%cd /content/mmselfsup

In [ ]:
%%writefile configs/selfsup/mae/mae_visualization.py
model = dict(
    type='MAE',
    backbone=dict(type='MAEViT', arch='l', patch_size=16, mask_ratio=0.75),
    neck=dict(
        type='MAEPretrainDecoder',
        patch_size=16,
        in_chans=3,
        embed_dim=1024,
        decoder_embed_dim=512,
        decoder_depth=8,
        decoder_num_heads=16,
        mlp_ratio=4.,
    ),
    head=dict(type='MAEPretrainHead', norm_pix=True, patch_size=16))

img_norm_cfg = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# dataset summary
data = dict(
    test=dict(
        pipeline = [
            dict(type='Resize', size=(224, 224)),
            dict(type='ToTensor'),
            dict(type='Normalize', **img_norm_cfg),]
    ))

### 5.3 可视化 MAE 模型恢复被遮挡图片的效果

我们可以使用工具 [tools/misc/mae_visualization.py](https://github.com/open-mmlab/mmselfsup/blob/dev/tools/misc/mae_visualization.py) 来可视化 MAE 模型恢复被遮挡图片的效果。

```shell
python tools/misc/mae_visualization.py ${IMG_PATH} ${CONFIG} ${CHECKPOINT_PATH} --device ${DEVICE}
```

参数：
+ IMG_PATH：测试图片路径
+ CONFIG：自监督训练的配置文件所在路径
+ CHECKPOINT_PATH：自监督预训练过程中保存下来（名为 `epoch_*.pth`）的模型文件路径
+ device：设置加载模型的设备，默认值为 'cuda:0'

**注意**：为了方便大家直接将训练好的模型迁移到下游任务上进行后续的训练，MMSelfSup 的 [模型库](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/model_zoo.md) 里的模型都是已经提取过 backbone 部分的权值文件，所以是没有 decoder 部分的权值，不可以拿来进行恢复被遮挡图片效果的可视化。

In [ ]:
!python tools/misc/mae_visualization.py \
data/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG \
configs/selfsup/mae/mae_visualization.py \
checkpoints/mae_visualize_vit_large.pth